In [1]:
import pandas as pd
import ollama
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:

def create_embeddings(l):
    res = ollama.embed(
        model="qllama/bge-large-en-v1.5:q4_k_m",
        input=l
    )

    vector = res["embeddings"]
    return vector



In [3]:
# s = "my name is keyur"
# print(create_embeddings(s))

In [4]:
# l=["hi","by"]
# print(len(create_embeddings(l)))


In [ ]:

jsons = os.listdir("jsons")
my_dict = []
chunk_id =0
for json_file in jsons:
    with open(f"jsons/{json_file}") as f:
        content = json.load(f)
    print(f"creating embeddings for {json_file}")
    embeddings =create_embeddings([c['text'] for c in content["chunks"]])
    for i,chunk  in enumerate(content["chunks"]):
        chunk["chunk_id"] = chunk_id
        chunk["embeddings"] = embeddings[i]
        chunk_id += 1
        my_dict.append(chunk)
        
    
# print(my_dict)

creating embeddings for 01_Introduction to Programming .json
creating embeddings for 02_Some Amazing Python Programs .json
creating embeddings for 03_Modules and Pip .json
creating embeddings for 04_Our First Python Program .json
creating embeddings for 05_Comments.json


In [ ]:

df = pd.DataFrame.from_records(my_dict)
# df.to_pickle("data.pkl")
df = pd.read_pickle("data.pkl")
# df = pd.read_csv("embed_data.csv")    #we are not using csv cause it stored embeddings as list hence we didnt want that so were using pkl to store

In [39]:
df["embeddings"].dtype

dtype('O')

In [44]:
print(np.vstack(df['embeddings']).shape)

(1678, 1024)


In [54]:
incoming_query = input("enter a query :")
question_embedding = create_embeddings(incoming_query)[0]
# print(np.vstack(df['embeddings']).shape)
similarities  =cosine_similarity(np.vstack(df['embeddings']),[question_embedding]).flatten()
top_results = 3
max_idx = similarities.argsort()[::-1][0:top_results]

In [ ]:
new_df  = df.loc[max_idx]
print(new_df[["number","title","text"]])    

     number                          title  \
22       01   Introduction to Programming    
1147     05                       Comments   
363      02  Some Amazing Python Programs    

                                       text  
22     When he was working on this language  
1147                  what program he wrote  
363         When I was introduced to python  
